## McMaster University Comp Sci 4TB3/6TB3, Winter Term 2016/17 — Lab 12
#### For the Labs on April 4 - April 6 <br>Due Thursday, April 6, 11 pm<br>Emil Sekerinski, Shucai Yao, Spencer Park<br>out of 24 points; extra points count towards any other lab

To be submitted in the folder Lab12 on GitLab; you should have received an invitation to join the project cs4tb3-winter17/yourmacid.
* You need to download the P0 jupyter notebook posted together with the lab, not use a previous version. You need to modify several notebooks, you should not create any other files.
* This assignment requires a jupyter installation. You can install it on your computer, use UTS labs, or use https://notebooks.azure.com/. _See the posts on Avenue about incompatibilities._
* In this lab, you are allowed to work in pairs, provided that you split the work equally and arrive at a common understanding of the solution. However, in that case you must state in your submission the person you worked with, such that similarities in the solution will not be construed as Academic Dishonesty. Working in groups of three or larger is not allowed and will be considered Academic Dishonesty. If you look for someone to work with, we will try to find a match, please contact the TAs.
* You are allowed and encouraged to talk to everyone in the course to get a common understanding of the problem, but you can share partial solutions only with your collaborator, if you work in a pair. The final submission must be your own, that is, you cannot submit identical submissions with two names on them.
* In the lab sessions, the solution to last week's lab questions are discussed and you can get help with this week's lab questions. Attendance at the labs is not checked.

### Lab Question 1 (Earley's Parser) [24 points]
Write a function `parse(g: "grammar", x: "input")` that returns true if input string `x` is according to grammar `g`. The grammar is a sequence (or tuple) of rules and each rule is a string. Terminals and nonterminals consist of a single character. Uppercase letters are, by convention, nonterminals, everything else is a terminal, except that `→` separates the left and right sides. The grammar has to be in plain BNF, without parenthesis or bars (`|`). For each nonterminal, there can be several rules. The order of the rules is irrelevant, except that the first nonterminal of the first rule is the start symbol. The input `x` is a string without leading or trailing space or separators in between. 

In [131]:
def parse(g: "grammar", x: "input"):  
    # init
    length = len(x)
    
    S = [Ordered_Set() for i in range(length + 1)]
    
    grammar = []
    for r in g:
        grammar.append(Rule(r[0], r[2:]))
    
    S[0].add(State(grammar[0],0,0))
    
    x = "^" + x + "$"
    
    #parse
    for i in range(length+1):
        for state in S[i]:
            
            ######Completer
            if state.e >= len(state.r.rhs):
                for old in S[state.j]:
                    if old.e < len(old.r.rhs) and state.r.lhs == old.r.rhs[old.e]:
                        S[i].add(State(old.r,old.e+1,old.j))
            
            else:
                expected = state.r.rhs[state.e]
                
                #######Predicter
                if 65 <= ord(expected) <= 90: 
                    for rule in grammar:
                        if rule.lhs == expected:
                            S[i].add(State(rule, 0, i))
                
                ######Scanner
                elif (expected == x[i+1]): 
                    S[i+1].add(State(state.r,state.e+1,state.j))
    
    if State(grammar[0], 1, 0) in S[length]: return True
    return False

# Custome Classes for Rule, State, and Ordered_Set

class Rule:
    def __init__(self, lhs, rhs):
        self.lhs = lhs
        self.rhs = rhs
    
    def __repr__(self):
        return str(self.lhs) + "→" + str(self.rhs)
    
    def __eq__(self, other):
        if isinstance(other, Rule):
            return self.lhs == other.lhs and self.rhs == other.rhs
        else: return False
        
    def __ne__(self, other):
        return not self.__eq__(other)

class State:
    def __init__(self, rule, expected, index):
        self.r = rule
        self.e = expected
        self.j = index
    
    def __repr__(self):
        return '('+str(self.r)+','+str(self.e)+','+str(self.j)+')'
    
    def __eq__(self, other):
        if isinstance(other, State):
            return self.r == other.r and self.e == other.e and self.j == other.j
        else: return False
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
class Ordered_Set(list):
    def add(self, item):
        if item not in self:
            self.append(item)

For example, this call should return `True`:

In [132]:
grammar = ("S→E", "E→T", "E→E+T", "T→F", "T→T*F", "F→a")

In [133]:
parse(grammar, "a*a+a")

True

You are free and encouraged to consult other resources on Earley's algorithm, in particular the original publication. However, you need to cite all the resources you used and you cannot copy an implementation you found elsewhere. This is unlikely going to help anyway: with the specifics of this question, an implementation can be written in about 20 lines, other implementations you can find are way more involved. Hints:

* Stick as close as possible to the formulation in the course notes.
* You do not need to check the well-formedness of the grammar.
* The algorithm requires that `x[n + 1]` is `$`, or any other character that does not appear in the input, and the input starts at `x[1]`. Simply prepend `^` (or any other character) and append `$` to the input.
* Represent an Earley item `A → σ · ω, j` by the tuple `(A, σ, ω, j)`.
* The algorithm requires a "fixed point computation" until the set `s[i]` does not change. One way is to introduce a set of items that still need to be visited and to continue the loop until that set is empty.
* During development only, output the list `s` of sets and check if that corresponds to the one in the course notes.

### Bonus Question (Earley's Parser) [12 points]
Modify the parser to output the set of all parse trees, in some suitable form, instead of true or false.